**Import Libraries**

In [ ]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import pandas as pd
import os
import collections
import re
import string
!pip install emoji
import emoji
from emoji import UNICODE_EMOJI

Using TensorFlow backend.


Import Dataset

In [ ]:
#path = "emotions1 - emotions1.csv"
path = 'Dataset_With_Emoji.csv'
df1 = pd.read_csv(path)

**In case we want to convert emojis to words, to include them in classification**

In [ ]:
"""def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)"""

'def is_emoji(s):\n    count = 0\n    for emoji in UNICODE_EMOJI:\n        count += s.count(emoji)\n        if count > 1:\n            return False\n    return bool(count)'

In [ ]:
"""def emoji2text(title):
    stemmed = []
    tokenized = []
    series = []
    print(type(series))
    for i in title:
      #print(type(title))
      for c in i:
        if is_emoji(c) == True:
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', ' ')
          #word2 = " ".join(word1)
          i = i.replace(c, word1)
      series.append(i)
    return series"""

'def emoji2text(title):\n    stemmed = []\n    tokenized = []\n    series = []\n    print(type(series))\n    for i in title:\n      #print(type(title))\n      for c in i:\n        if is_emoji(c) == True:\n          word1 = emoji.demojize(c)\n          word1 = word1.replace(\'_\', \' \')\n          word1 = word1.replace(\':\', \' \')\n          #word2 = " ".join(word1)\n          i = i.replace(c, word1)\n      series.append(i)\n    return series'

In [ ]:
"""df1.Tweet = emoji2text(df1.Tweet)
tweets = (df1['Tweet'])
df1.head()
labels = df1['Affect Dimension']"""

"df1.Tweet = emoji2text(df1.Tweet)\ntweets = (df1['Tweet'])\ndf1.head()\nlabels = df1['Affect Dimension']"

In [ ]:
tweets = (df1['Tweet'])
labels = df1['Affect Dimension']

**For One-Hot Encoding**

In [ ]:
encoded_labels = []
for label in labels:
    if label == 'anger':
        encoded_labels.append(0)
    elif label == 'joy':
        encoded_labels.append(1)
    elif label == 'neutral':
        encoded_labels.append(2)
    elif label == 'sadness/fear':
        encoded_labels.append(3)
df1['Affect Dimension']=np.asarray(encoded_labels)

In [ ]:
df1['Affect Dimension'].unique()

df1.shape

(12822, 4)

In [ ]:
joy = []
sadness = []
neutral = []
anger = []
for l in df1['Affect Dimension']:
    if l == 0:
        anger.append(1)
        joy.append(0)
        neutral.append(0)
        sadness.append(0)
    elif l == 1:
        anger.append(0)
        joy.append(1)
        neutral.append(0)
        sadness.append(0)
    elif l == 2:
        anger.append(0)
        joy.append(0)
        neutral.append(1)
        sadness.append(0)
    elif l == 3:
        anger.append(0)
        joy.append(0)
        neutral.append(0)
        sadness.append(1)

In [ ]:
df1['anger']= anger
df1['joy']= joy
df1['sadness'] = sadness
df1['neutral'] = neutral

In [ ]:
df1.head()

,Unnamed: 0,Tweet,Affect Dimension,Unnamed: 2,anger,joy,sadness,neutral
0,0,pouting face @chetan_bhagat face with steam ...,0,NaN,1,0,0,0
1,1,Low key Dro f*cked Molly like he had a point t...,0,NaN,1,0,0,0
2,2,@TeleTransOne @eleganza_lily @FemFataleXX Tha...,0,NaN,1,0,0,0
3,3,@F3Alpha #F3Roswell #Preblast\nI've face with...,0,NaN,1,0,0,0
4,4,Did they offend us and they want it to sound n...,0,NaN,1,0,0,0


**Preprocessing of Dataset**

In [ ]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

df1['Text_Clean'] = df1['Tweet'].apply(lambda x: remove_punct(x))

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in df1.Text_Clean]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [ ]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]

In [ ]:
result = [' '.join(sen) for sen in filtered_words]

In [ ]:
df1['Text_Final'] = result

In [ ]:
df1['tokens'] = filtered_words

In [ ]:
df1 = df1[['Text_Final', 'tokens', 'Affect Dimension', 'anger', 'joy', 'neutral', 'sadness']]

In [ ]:
df1.head()

,Text_Final,tokens,Affect Dimension,anger,joy,neutral,sadness
0,pouting face chetanbhagat face steam nose face...,"[pouting, face, chetanbhagat, face, steam, nos...",0,1,0,0,0
1,low key dro fcked molly like point prove lmao ...,"[low, key, dro, fcked, molly, like, point, pro...",0,1,0,0,0
2,teletransone eleganzalily femfatalexx thats li...,"[teletransone, eleganzalily, femfatalexx, that...",0,1,0,0,0
3,f3alpha f3roswell preblast\nive face steam nos...,"[f3alpha, f3roswell, preblast\nive, face, stea...",0,1,0,0,0
4,offend us want sound new,"[offend, us, want, sound, new]",0,1,0,0,0


**Splitting the dataset into train and test**

In [ ]:
data_train, data_test = train_test_split(df1, test_size=0.20, random_state=42)

In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

115533 words total, with a vocabulary size of 22463
Max sentence length is 153


In [ ]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))


28841 words total, with a vocabulary size of 9114
Max sentence length is 51


**Importing Google News word vectors**

In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-06-30 20:56:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.134.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.134.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  31.8MB/s    in 50s     

2020-06-30 20:57:38 (31.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [ ]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Functions for using word2vec**

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [ ]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)
len(training_embeddings)


In [ ]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300


**Using Tokenizer**

In [ ]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

In [ ]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Defining the Convolution Net

In [ ]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [ ]:
label_names = ['anger', 'joy', 'neutral', 'sadness']
y_train = data_train[label_names].values
#print(type(y_train))
print(y_train.shape)

In [ ]:
x_train = train_cnn_data
print(x_train.shape)
y_tr = y_train

In [ ]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))
model.summary()

**Defining parameters**

In [ ]:
num_epochs = 3
batch_size = 32

**Training the model**

In [ ]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

**Testing the accuracy on Test Set**

In [ ]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=2)

In [ ]:
labels = [0, 1, 2, 3]

In [ ]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [ ]:
sum(data_test['Affect Dimension']==prediction_labels)/len(prediction_labels)